# Using SageMaker Model Parallel 

SageMaker Distributed Model Parallel ("SDMP") is a feature-rich propriterary library that implements model parallelism and hybrid parallelism and is optimized for the SageMaker infrastructure. It supports the TensorFlow and PyTorch frameworks and allows you to automatically partition models between devices with minimal code changes to your training script.

In this example we will develop a hybrid parallel job using the SDMP library. We will reuse the example of training image classification model for Hymenoptera dataset. For this, we will use Resnet18 model. Usually CV models (such as Resnet18) can fit into a single GPU, and in this case, model parallelism is not required. However, we chose these model architecture and task for educational purposes only as it's easy to manage and quick to train for demo purposes.

We start with usual imports and data preparations.

In [ ]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = get_execution_role()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sm-modelparallel-distribution-options'
print('Bucket:\n{}'.format(bucket))

In [2]:
# Data preparation was already done in Chapter06/2_distributed_training_PyTorch.ipynb
# If you skipped it, then run following code below

! wget https://download.pytorch.org/tutorial/hymenoptera_data.zip
! unzip hymenoptera_data.zip
data_url = sagemaker_session.upload_data(path="./hymenoptera_data", key_prefix="hymenoptera_data")

## Configuring Model/Hybrid Parallelism

First, let’s understand how our training will be executed and how we can configure parallelism. For this, we will use the distribution object of the SageMaker training job. It has two config components: `modelparallel`and `mpi`. `distribution` configuration is provided as part of SageMaker Estimator.

```python
distribution={
                  "modelparallel": {...},
                   "mpi": {...}
```

Let's review in details available `modelparallel` and `mpi` parameters.

### `modelparallel` config

The `modelparallel` object defines the configuration of the SDMP library. Then, in the code snippet, we set 2-way model parallelism (the `partitions` parameter is set to 2). Also, we enable hybrid parallelism by setting `ddp=True`. When data parallelism has been enabled, SDMP will automatically infer the data parallel size based on the number of training processes and the model parallelism size. Another important parameter is `auto_partition`, so SDMP automatically partitions the model between GPU devices.

```python
"modelparallel": {
    "enabled":True,
    "parameters": {
        "microbatches": 8, # The number of microbatches to perform pipelining over. 1 means no pipelining. Batch size must be divisible by the number of microbatches.
                            # A microbatch is a smaller subset of a given training mini-batch. The pipeline schedule determines which microbatch is executed by which device for every time slot.   
        "placement_strategy": "cluster", # more advanced topic: https://sagemaker.readthedocs.io/en/stable/api/training/smd_model_parallel_general.html#placement-strategy-with-tensor-parallelism 
        "pipeline": "interleaved",
        "optimize": "speed", 
        "partitions": 2,
        "auto_partition": True,
        "ddp": True, # enables hybrid parallelism: model + data. Makes sense in multi-GPU nodes only.
    }
}
```
### `mpi` config

SageMaker relies on the `mpi` utility to run distributed computations. In the following code snippet, we set it to run 8 training processes. Here, `processes_per_host` defines how many training processes will be run per host, which includes both processes running model parallelism, data parallelism, or tensor parallelism. In most cases, the number of processes should match the number of available GPUs in the node.

```python
"mpi": {
    "enabled": True, # must be enabled
    "processes_per_host": 1, # Pick your processes_per_host
    "custom_mpi_options": mpioptions 
},
```

Now, let's review how we need to modify training script to use SDMP library.


## Modifying Training Script

One of the benefits of SDMP is that it requires minimal changes to your training script. This is achieved by using the Python decorator to define computations that need to be run in a model parallel or hybrid fashion. Additionally, SDMP provides an API like other distributed libraries such as Horovod or PyTorch DDP. Below we review some key modifications we need to make. Full sources is available here: `4_sources/train_sm_mp.py`.

1. We start by importing and initializing the SDMP library:

```python
    import smdistributed.modelparallel.torch as smp
    smp.init()
```

2. SDMP manages the assignment of model partitions to the GPU device, and you don’t have to explicitly move the model to a specific device (in a regular PyTorch script, you need to move the model explicitly by calling the model.to(device) method). In each training script, you need to choose a GPU device based on the SMDP local rank:

```python
    torch.cuda.set_device(smp.local_rank())
    device = torch.device("cuda")
```

3. Next, we need to wrap the PyTorch model and optimizers in SDMP implementations. This is needed to establish communication between the model parallel and data parallel groups. Once wrapped, you will need to use SDMP-wrapped versions of the model and optimizer in your training script. Note that you still need to move your input tensors (for instance, data records and labels) to this device using the PyTorch input_tensor.to(device) method:

```python
    model = smp.DistributedModel(model)
    optimizer = smp.DistributedOptimizer(optimizer)
```

4. After that, we need to configure our data loaders. SDMP doesn’t have any specific requirements for data loaders, except that you need to ensure batch size consistency. It’s recommended that you use the `drop_last=True` flag to enforce it. This is because, internally, SDMP breaks down the batch into a set of micro-batches to implement pipelining. Hence, we need to make sure that the batch size is always divisible by the micro-batch size. Note that, in the following code block, we are using the SDMP API to configure a distributed sampler for data parallelism:

```python
    dataloaders_dict = {}
    train_sampler = torch.utils.data.distributed.DistributedSampler(
        image_datasets["train"], num_replicas=sdmp_args.dp_size, rank=sdmp_args.dp_rank)

    dataloaders_dict["train"] = torch.utils.data.DataLoader(
        image_datasets["train"],
        batch_size=args.batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        sampler=train_sampler,
        drop_last=True,)

    dataloaders_dict["val"] = torch.utils.data.DataLoader(
        image_datasets["val"],
        batch_size=args.batch_size,
        shuffle=False,
        drop_last=True,)
```

5. Once we have our model, optimizer, and data loaders configured, we are ready to write our training and validation loops. To implement model parallelism, SDMP provides a @smp.step decorator. Any function decorated with @smp.set splits executes internal computations in a pipelined manner. In other words, it splits the batch into a set of micro-batches and coordinates the computation between partitions of models across GPU devices. Here, the training and test computations are decorated with @smp.step. Note that the training step contains both forward and backward passes, so SDMP can compute gradients on all partitions. We only have the forward pass in the test step:

```python
    @smp.step
    def train_step(model, data, target, criterion):
        output = model(data)
        loss = criterion(output, target)
    model.backward(loss)  #  instead of PyTorch loss.backward()
    return output, loss

    @smp.step
    def test_step(model, data, target, criterion):
        output = model(data)
        loss = criterion(output, target)
        return output, loss        
```

6. We use decorated training and test steps in the outer training loop as follows. The training loop construct is like a typical PyTorch training loop with one difference. Since SDMP implements pipelining over micro-batches, the loss values will be calculated for micro-batches, too (that is, the `loss_mb` variable). Hence, to calculate the average loss across the full batch, we call the `reduce_mean()` method. Note that all variables returned by the `@smp.step` decorated function are instances of the class that provides a convenient API to act across mini-batches (such as the `reduce_mean()` or `concat()` methods):

```python
    for epoch in range(num_epochs):
            for phase in ["train", "val"]:
            if phase == "train":
                model.train()  # Set model to training
            else:
                model.eval()  # Set model to evaluate
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase ==
                    if phase == "train":
                        outputs, loss_mb = train_step(model, inputs, labels, criterion)
                        loss = loss_mb.reduce_mean()
                        optimizer.step()
                    else:
                        outputs, loss_mb = test_step(model, inputs, labels, criterion)
                        loss = loss_mb.reduce_mean()
```

Feel free review full training script by running cell below:

In [ ]:
! pygmentize 4_sources/train_sm_mp.py

## Running SDMP Training Job

In the following code block, we configure our training job to run on 2 instances with a total of 16 GPUs in the training cluster. Our distribution object defines 2-way model parallelism. Since the total number of training processes is 16, SDMP will automatically infer 8-way data parallelism. In other words, we split our model between 2 GPU devices, and have a total of 8 copies of the model:

In [17]:

from sagemaker.pytorch import PyTorch

instance_type = 'ml.p3.16xlarge'
instance_count = 2

smd_mp_estimator = PyTorch(
          entry_point="train_sm_mp.py", # Pick your train script
          source_dir='4_sources',
          role=role,
          instance_type=instance_type,
          sagemaker_session=sagemaker_session,
          image_uri="763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.10.2-gpu-py38-cu113-ubuntu20.04-sagemaker",
          instance_count=instance_count,
          hyperparameters={
              "batch-size":64,
              "epochs":30,
          },
          disable_profiler=True,
          debugger_hook_config=False,
          distribution={
              "smdistributed": {
                  "modelparallel": {
                      "enabled":True,
                      "parameters": {
                          "microbatches": 8,
                          "placement_strategy": "cluster",
                          "pipeline": "interleaved",
                          "optimize": "speed", 
                          "partitions": 2,
                          "auto_partition": True,
                          "ddp": True,
                      }
                  }
              },
              "mpi": {
                    "enabled": True,
                    "processes_per_host": 1,
                    "custom_mpi_options": "-verbose -x orte_base_help_aggregate=0" 
              },
          },
          base_job_name="SMD-MP",
      )

In [ ]:
smd_mp_estimator.fit(inputs={"train":f"{data_url}/train", "val":f"{data_url}/val"})

## Summary
In the example above we implemented simple hybrid paralklelism using SDMP library. We encourage you to experiment with various SDMP configuration parameters (please refer to the distribution object from the previous section) to develop good intuition, such as:
- Change the number of model partitions. Our implementation has 2 partitions; you might try to set 1 or 4 partitions and see how this changes the data parallel and model parallel groups.
- Change the number of micro-batches and batch size to see how it impacts training speed. In production scenarios, you will likely need to explore an upper memory limit for batch size and micro-batches to improve training efficiency.
- See how the type of pipeline implementation (interleaved or simple) impacts the training speed.